# Установка зависимостей & импорт либр :)

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install git+https://github.com/evalplus/evalplus.git

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer
import torch, json
from tqdm import tqdm

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"видеокарта = {gpu_stats.name}. VRAM= {max_memory} GB.")

# Константы

In [ ]:
MODEL_NAME = "unsloth/Qwen3-0.6B-Base-bnb-4bit" # или любая другая 
ADAPTER_NAME = "dxnay/qwen3-0.6b-lora-tuned"  # hf
BENCHMARK = "openai_humaneval"
TRAIN_PATH = "/kaggle/input/qwen-data/train.jsonl"
OUTPUT_BASE = "samples_base.jsonl"
OUTPUT_TUNED = "samples_tuned.jsonl"
MAX_NEW_TOKENS = 512
MAX_SEQ_LENGTH = 2048
SEED = 3407

# Инициализация модельки

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)


# конфигурация lora параметров

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.07,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    use_rslora=False,
    loftq_config=None,
)

## преобработка данных

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")

def formatting_prompts_func(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = load_dataset("json", data_files=TRAIN_PATH, split="train")
dataset = dataset.map(lambda x: {"conversations": x["messages"]})
dataset = dataset.map(formatting_prompts_func, batched=True)


# трейн модели 

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=3,  
    warmup_ratio=0.05,
    learning_rate=2e-5,
    max_grad_norm=0.9,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine", 
    seed=SEED,
    output_dir="outputs",
    report_to="none",
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    dataset_num_proc=4,
    packing=False,
    args=training_args,
)

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n"
)

trainer_stats = trainer.train()

In [ ]:
print(f"Ушло времени на обучение: {trainer_stats.metrics['train_runtime']} секунд.")

# Сохранение tuned qwen

In [ ]:
model.save_pretrained(ADAPTER_NAME)
tokenizer.save_pretrained(ADAPTER_NAME)

# Функция для генерации на humaneval

In [ ]:
def generate_samples(model, tokenizer, output_path: str, benchmark: str, benchmark_type: str):
    tokenizer.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    model.eval()
    dataset = load_dataset(benchmark, split="test")

    def extract_function(text):
        lines = text.strip().split("\n")
        try:
            start = next(i for i, l in enumerate(lines) if l.strip().startswith("def "))
        except StopIteration:
            return text
        func = [lines[start]]
        for l in lines[start+1:]:
            if l.startswith(" ") or l.startswith("\t") or not l.strip():
                func.append(l)
            else:
                break
        return "\n".join(func)

    with open(output_path, "w", encoding="utf-8") as f:
        for i, item in enumerate(tqdm(dataset)):
            if benchmark_type == "humaneval":
                task_id = f"HumanEval/{i}"
                prompt = item["prompt"]
            elif benchmark_type == "mbpp":
                task_id = f"MBPP/{i}"
                prompt = item["text"]
            else:
                raise ValueError("Неверный тип бенчмарка")

            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                output = model.generate(
                    **inputs,
                    max_new_tokens=MAX_NEW_TOKENS,
                    do_sample=False
                )

            decoded = tokenizer.decode(output[0], skip_special_tokens=True)
            func_code = extract_function(decoded[len(prompt):])
            f.write(json.dumps({"task_id": task_id, "completion": func_code}) + "\n")

    print(f"Сэмплы сохранены в {output_path}")


# Оценка на HumanEval+MBPP (pretrained)

In [ ]:
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")


In [ ]:
generate_samples(base_model, tokenizer, "samples_base_humaneval.jsonl", "openai/openai_humaneval", "humaneval")
!evalplus.evaluate --dataset humaneval --samples samples_base_humaneval.jsonl


In [ ]:
generate_samples(base_model, tokenizer, "samples_base_mbpp.jsonl", "Muennighoff/mbpp", "mbpp")
!evalplus.evaluate --dataset mbpp --samples samples_base_mbpp.jsonl


# Оценка на HumanEval+MBPP (tuned)

In [ ]:
tuned_model, _ = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    adapter_name=ADAPTER_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

In [ ]:
generate_samples(tuned_model, tokenizer, "samples_tuned_humaneval.jsonl", "openai/openai_humaneval", "humaneval")
!evalplus.evaluate --dataset humaneval --samples samples_tuned_humaneval.jsonl


In [ ]:
generate_samples(tuned_model, tokenizer, "samples_tuned_mbpp.jsonl", "Muennighoff/mbpp", "mbpp")
!evalplus.evaluate --dataset mbpp --samples samples_tuned_mbpp.jsonl


# для пуша на хаб 

In [ ]:
from huggingface_hub import login, HfApi
token = ''
login(token=token)

In [ ]:
model.push_to_hub(ADAPTER_NAME, private=False)
tokenizer.push_to_hub(ADAPTER_NAME, private=True)
